# Creating a Shooting Game with Python

I refered to this Youtube Video to make a shooting game with Python.
https://www.youtube.com/watch?v=-e_5sOsKqrU&list=PLJTV6uzKdfpd1wfjkhXp_Lmtg3NZyz6zl&index=2

In [ ]:
# Game screen composition

import pygame
import sys
from time import sleep
import random

padWidth = 480   # the size of the game screen will be used below
padHeight = 640
rockImage = ['rock01.png','rock02.png','rock03.png','rock04.png','rock05.png', \
            'rock06.png','rock07.png','rock08.png','rock09.png','rock10.png', \
            'rock11.png','rock12.png','rock13.png','rock14.png','rock15.png', \
            'rock16.png','rock17.png','rock18.png','rock19.png','rock20.png', \
            'rock21.png','rock22.png','rock23.png','rock24.png','rock25.png',\
            'rock26.png','rock27.png','rock28.png','rock29.png','rock30.png']
explosionSound = ['explosion01.wav', 'explosion02.wav', 'explosion03.wav', 'explosion04.wav'] # gamesound

# score how many rocks the fighter get
def writeScore(count):
    global gamePad
    font = pygame.font.Font('NanumGothic.ttf', 20)
    text = font.render('파괴한 운석 수: ' + str(count), True, (255, 255, 255)) # the color is white
    gamePad.blit(text, (10,0)) # the left upper corner
    
def writePassed(count):
    global gamePad
    font = pygame.font.Font('NanumGothic.ttf', 20)
    text = font.render('놓친 운석 수: ' + str(count), True, (255, 0, 0)) # the color is red
    gamePad.blit(text, (350,0)) # the right upper corner  

def writeMessage(text):
    global gamePad, gameOverSound
    textfont = pygame.font.Font('NanumGothic.ttf', 60) # the font of the message
    text = textfont.render(text, True, (255,0,0)) # the color = red
    textpos = text.get_rect() # the position of the text
    textpos.center = (padWidth/2, padHeight/2) # the position will be in the middle
    gamePad.blit(text, textpos) # show the message on the screen
    pygame.display.update()
    pygame.mixer.music.stop()
    gameOverSound.play()
    sleep(2)
    pygame.mixer.music.play(-1)
    runGame()
    
def crash():
    global gamePad
    writeMessage('전투기 파괴!')
    
def gameOver():
    global gamePad
    writeMessage('게임 오버!')
    
def drawObject(obj, x, y): # drawing the objects shown in the game
    global gamePad
    gamePad.blit(obj, (x, y)) # take the background surface and draw it onto the screen and position it at (x,y)    

def initGame(): # set the game for the first
    global gamePad, clock, background, fighter, missile, explosion, missileSound, gameOverSound # global variables
    pygame.init() # reset the library, pygame
    gamePad = pygame.display.set_mode((padWidth, padHeight)) # set the size
    pygame.display.set_caption("Youjin\'sShooting") # the name of the game
    background = pygame.image.load("background.png") # background 
    fighter = pygame.image.load('fighter.png') # fighter icon
    missile = pygame.image.load('missile.png') # missile icon
    explosion = pygame.image.load('explosion.png')
    pygame.mixer.music.load('music.wav')
    pygame.mixer.music.play(-1)
    missileSound = pygame.mixer.Sound('missile.wav')
    gameOverSound = pygame.mixer.Sound('gameover.wav')
    clock = pygame.time.Clock() 
    
def runGame(): # this funtion runs the game
    global gamePad, clock, background, fighter, missile, explosion, missileSound
    
    # the size of the fighter
    fighterSize = fighter.get_rect().size
    fighterWidth = fighterSize[0]
    fighterHeight = fighterSize[1]
    
    # the first location of the fighter
    x = padWidth * 0.45
    y = padHeight * 0.9
    fighterX = 0
    
    missileXY = []
    
    # make the rocks randomly appear 
    rock = pygame.image.load(random.choice(rockImage))
    rockSize = rock.get_rect().size # the size of the rocks
    rockWidth = rockSize[0]
    rockHeight = rockSize[1]
    destroySound = pygame.mixer.Sound(random.choice(explosionSound))
    
    # the first location of the rock
    rockX = random.randrange(0, padWidth - rockWidth)
    rockY = 0
    rockSpeed = 2
    
    # when a rock gets to the fighter = True
    isShot = False
    shotCount = 0 # when missing a rock
    rockPassed = 0
    
    onGame = False
    while not onGame:
        for event in pygame.event.get():
            if event.type in [pygame.QUIT]: # if you close the game prompt
                pygame.quit() # it ends the game
                sys.exit() # and close the system
                
            if event.type in [pygame.KEYDOWN]: # when a key on a keyboard is pressed
                if event.key == pygame.K_LEFT: # when the left key is pressed
                    fighterX -= 5 # move the fighter to the left
                
                elif event.key == pygame.K_RIGHT: # when the right key is pressed
                    fighterX += 5 # move the fighter to the right
                    
                elif event.key == pygame.K_SPACE: # when the space key is pressed
                    missileSound.play() # missile sound
                    missileX = x + fighterWidth/2 # the first location of the missile(horizonal)  = missile will be shoot in the middle of the fighter icon 
                    missileY = y - fighterHeight # the first location of the missile(vertical) = missile will be shoot where the fighter is
                    missileXY.append([missileX, missileY]) # save the x, y values in the list variable
                    
            if event.type in [pygame.KEYUP]: # when a key is done with being pressed
                if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                    fighterX = 0
                
        drawObject(background, 0, 0) # make the background color black
        
        # relocate the fight after pressing keys
        x += fighterX
        if x < 0:
            x = 0 # it can't go outside the screen
        elif x > padWidth - fighterWidth:
            x = padWidth - fighterWidth # it can't go outside the screen
        
        # when the rock kills the fighter
        if y < rockY + rockHeight:
            if (rockX > x and rockX < x + fighterWidth) or (rockX + rockWidth > x and rockX + rockWidth < x + fighterWidth):
                crash()
        
        drawObject(fighter, x, y) # draw the fighter in the middle, bottom of the screen
        
        # locate the missile when the key is pressed
        if len(missileXY) != 0: # when the list variable(missileXY) has any value
            for i, bxy in enumerate(missileXY): # redraw the missiles on the screen
                bxy[1] -= 10 # the missile seems to be shoot toward the top of the screen
                missileXY[i][1] = bxy[1] # missile location has to be set to bxy[1]
                
                # when the missile shoot the rock
                if bxy[1] < rockY:
                    if bxy[0] > rockX and bxy[0] < rockX + rockWidth: # if the location of the missile is same as the one of the rock
                        missileXY.remove(bxy) # remove the missile
                        isShot = True
                        shotCount += 1
                
                if bxy[1] <= 0: # when the missile goes outside of the screen
                    try:
                        missileXY.remove(bxy) # remove the missile
                    except:
                        pass
                    
        if len(missileXY) != 0:
            for bx, by in missileXY:
                drawObject(missile, bx, by) # draw the missiles
                
        writeScore(shotCount)
                
        rockY += rockSpeed # change the rock location 
        
        # when the rock gets to the earth(bottom of the screen)
        if rockY > padHeight:
            # new rock(random)
            rock = pygame.image.load(random.choice(rockImage))
            rockSize = rock.get_rect().size
            rockWidth = rockSize[0]
            rockHeight = rockSize[1]
            rockX = random.randrange(0, padWidth - rockWidth)
            rockY = 0
            rockPassed += 1
        
        if rockPassed == 3:
            gameOver()
        
        writePassed(rockPassed)
            
        # when get the rock
        if isShot:
            # the rock explodes
            drawObject(explosion, rockX, rockY) # the image of explosion
            destroySound.play()
            
            rock = pygame.image.load(random.choice(rockImage))
            rockSize = rock.get_rect().size
            rockWidth = rockSize[0]
            rockHeight = rockSize[1]
            rockX = random.randrange(0, padWidth - rockWidth)
            rockY = 0
            destroySound = pygame.mixer.Sound(random.choice(explosionSound))
            isShot = False
            
            rockSpeed += 0.2 # speed up the rock
            if rockSpeed >= 10: # when the speed gets to the maximum speed
                rockSpeed = 10 # keep it as the maximum speed
        
        drawObject(rock, rockX, rockY)
        
        pygame.display.update() # redisplay the game 
        
        clock.tick(60) # the number of frames per second = 60
        
    pygame.quit()
    
initGame()
runGame()
    